# Multilingual scoring

Score verified Reddit posts using multilingual sentiment (XLM-RoBERTa) and a simple engagement metric.

In [1]:
import numpy as np
import pandas as pd
import duckdb
from pathlib import Path
from transformers import pipeline


/Users/ivantyshchenko/Projects/Python/DataPatron/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data from SQL

Query verified submissions plus their comments directly into pandas using DuckDB.

In [ ]:
%%sql
-- Створення нової аналітичної таблиці
CREATE TABLE main.dataset_posts_with_comments AS
SELECT
    s.id AS post_id,
    s.author AS post_author,
    s.created_at AS post_created_at,
    s.subreddit,
    s.title,
    s.selftext,
    s.score AS post_score,
    s.upvote_ratio,
    s.num_comments,
    s.url,
    s.domain,
    ARRAY_AGG(c.body) FILTER (WHERE c.body IS NOT NULL) AS comments_content,
    ARRAY_AGG(c.score) FILTER (WHERE c.score IS NOT NULL) AS comments_scores
FROM
    main.submissions_final_verified s
LEFT JOIN
    main.comments c ON s.id = c.submission_id
GROUP BY
    s.id,
    s.author,
    s.created_at,
    s.subreddit,
    s.title,
    s.selftext,
    s.score,
    s.upvote_ratio,
    s.num_comments,
    s.url,
    s.domain;

In [3]:
DB_PATH = Path('/Users/ivantyshchenko/Projects/Python/DataPatron/data/reddit.db')

QUERY = '''
SELECT * FROM dataset_posts_with_comments
'''

with duckdb.connect(str(DB_PATH)) as conn:
    posts_df = conn.execute(QUERY).df()

print(f'Loaded {len(posts_df)} posts')


Loaded 3702 posts


## Set up multilingual sentiment model

XLM-RoBERTa sentiment model with full-score output for downstream scoring.

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

def preprocess(text):
  parts = []
  for t in text.split(" "):
      if t.startswith("@") and len(t) > 1:
          t = "@user"
      if t.startswith("http"):
          t = "http"
      parts.append(t)
  return " ".join(parts)

model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

text = preprocess("Good night 😊")
encoded = tokenizer(text, return_tensors="pt")
scores = model(**encoded).logits[0].detach().numpy()
scores = softmax(scores)

for idx in scores.argsort()[::-1]:
  print(f"{config.id2label[idx]} {scores[idx]:.4f}")

/Users/ivantyshchenko/Projects/Python/DataPatron/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'NoneType' object has no attribute 'endswith'

In [ ]:
MODEL_NAME = 'cardiffnlp/twitter-xlm-roberta-base-sentiment'

sentiment_pipeline = pipeline(
    'sentiment-analysis',
    model=MODEL_NAME,
    tokenizer=MODEL_NAME,
    top_k=None,
    truncation=True,
    max_length=512
)

TOXIC_KEYWORDS = {
    'fake', 'bot', 'propaganda', 'liar', 'stupid', 'idiot', 'scam',
    'фейк', 'бот', 'брехня', 'пропаганда', 'іпсо', 'дурень', 'ідіот',
    'вранье', 'ложь'
}


## Metric helpers

Plain functions for multilingual sentiment scoring, efficiency, and UCQS.

In [ ]:
def to_multilingual_score(text: str) -> float:
    if not isinstance(text, str) or not text.strip():
        return 0.0
    try:
        results = sentiment_pipeline(text)[0]
        scores = {item['label'].lower(): item['score'] for item in results}
        return scores.get('positive', 0.0) - scores.get('negative', 0.0)
    except Exception as exc:
        print(f'Sentiment fallback for text due to error: {exc}')
        return 0.0

def calculate_efficiency(post_score, num_comments, upvote_ratio) -> float:
    score = post_score if pd.notnull(post_score) else 0
    comments = num_comments if pd.notnull(num_comments) else 0
    ratio = upvote_ratio if pd.notnull(upvote_ratio) else 0.5
    raw_engagement = score + (comments * 2.0)
    log_engagement = np.log1p(raw_engagement)
    return round(log_engagement * ratio, 2)

def calculate_ucqs(comments_content) -> float:
    if not isinstance(comments_content, list) or not comments_content:
        return 50.0
    sentiments = []
    toxic_count = 0
    valid_comments = 0
    for text in comments_content:
        if not isinstance(text, str) or not text.strip():
            continue
        valid_comments += 1
        score = to_multilingual_score(text[:512])
        sentiments.append(score)
        lower = text.lower()
        if any(k in lower for k in TOXIC_KEYWORDS) or score < -0.7:
            toxic_count += 1
    if not sentiments:
        return 50.0
    s_avg = float(np.mean(sentiments))
    s_var = float(np.std(sentiments))
    t_ratio = toxic_count / valid_comments if valid_comments else 0
    ucqs = 50 + (s_avg * 40) - (s_var * 20) - (t_ratio * 30)
    return float(np.clip(ucqs, 0, 100))


## Apply metrics

Compute both scores and inspect the result set.

In [ ]:
posts_df['efficiency'] = posts_df.apply(
    lambda row: calculate_efficiency(row['post_score'], row['num_comments'], row['upvote_ratio']),
    axis=1
)

posts_df['ucqs'] = posts_df['comments_content'].apply(calculate_ucqs)

posts_df[['post_id', 'efficiency', 'ucqs']].head()
